In [1]:
import numpy as np 
import os
import pandas as pd
import geopandas as gpd #important
import rasterio
import matplotlib.pyplot as plt
import operator
import seaborn as sns
import psycopg2
import shapely 
import laspy #las open
from laspy.file import File as read_las #open las 
from shapely.geometry import Point #convert las to gpd
import descartes #map figures
import rtree #spatial merge
import statistics #median

In [2]:
#1. READ LAS#
in_las = read_las('lassample.las', mode='r')

In [3]:
#2. CONVERTING LAS TO PANDAS#

#Import LAS into numpy array (X=raw integer value; x=scaled float value)
lidar_points = np.array((in_las.x,in_las.y,in_las.z,in_las.intensity)).transpose()

#Transform to pandas DataFrame
lidar_df=pd.DataFrame(lidar_points, columns = ['X coordinate', 'Y coordinate', 'Z coordinate', 'Intensity'])

#Transform to geopandas GeoDataFrame
Geometry = [Point(xy) for xy in zip(in_las.x,in_las.y)]
lidar = gpd.GeoDataFrame(lidar_df, crs = {'init' :'epsg:32643'}, geometry=Geometry) #set correct spatial reference

In [4]:
#3. READ SHAPEFILE#
shapefile = gpd.read_file("shapefilesample.shp")

#make id the index
shapefile = shapefile.set_index(shapefile.id)
shapefile  = shapefile.drop(['id'], axis = 1)

In [5]:
#4. SPATIAL MERGE LIDAR AND SHAPEFILE BY GEOMETRY COLUMNS# 
#Keep building id (stored as index_right)
buildings = gpd.sjoin(lidar, shapefile) 

In [6]:
#5. CALCULATE MEDIAN HEIGHT OF BUILDING BY GROUPING BY BUILDING ID# 

building_height_input = buildings.groupby('index_right')['Z coordinate'].median() #add filters here if needed
building_height = pd.DataFrame(building_height_input, index = None )
building_height = building_height.rename_axis(index='ID').rename(columns={'Z coordinate':'Roof Height'})

In [7]:
#6. CREATE BUFFER FROM THE BUILDING EDGES#

#create 1 metre buffer
buffer_line1a = shapefile['geometry'].buffer(distance = 2) #returns Geoseries
buffer_line1 = gpd.GeoDataFrame(buffer_line1a, columns = ['geometry'], crs = {'init' :'epsg:32643'}) #convert to Geodatabase

#create 2 metre buffer
buffer_line2a = shapefile['geometry'].buffer(distance = 1)
buffer_line2 = gpd.GeoDataFrame(buffer_line2a, columns = ['geometry'], crs = {'init' :'epsg:32643'})

#Take difference between two buffers
buffer_line = buffer_line1['geometry'].difference(buffer_line2)
buffer_line = gpd.GeoDataFrame(buffer_line, columns = ['geometry'], crs = {'init' :'epsg:32643'})

In [8]:
#7. SPATIAL MERGE LIDAR AND BUFFER_LINE BY GEOMETRY COLUMNS# 
#Keep building id (stored as index_right)
border = gpd.sjoin(lidar, buffer_line)

In [9]:
#8. CALCULATE GROUND LEVEL OF LIDAR Z POINTS IN BORDER BY GROUPING BY BUILDING ID #
#(version with 0.025 quantile taken as the parameter)

ground_height_input = border.groupby('index_right')['Z coordinate'].quantile(q=0.025)
#if (ground_height_input)
ground_height = pd.DataFrame(ground_height_input)
ground_height = ground_height.rename_axis(index='ID').rename(columns={'Z coordinate':'Ground Level'})

In [10]:
#8. CALCULATE GROUND LEVEL OF LIDAR Z POINTS IN BORDER BY GROUPING BY BUILDING ID#
#(version with minimum taken as the parameter)

ground_height_input2 = border.groupby('index_right')['Z coordinate'].min()
ground_height2 = pd.DataFrame(ground_height_input2)
ground_height2 = ground_height2.rename_axis(index='ID').rename(columns={'Z coordinate':'Ground Level'})

In [10]:
#9. CREATE DATAFRAME OF BUILDING ID AND BUILDING HEIGHT#

height_difference1 = building_height['Roof Height'] - ground_height['Ground Level']
#height_difference.dropna(how = 'any', inplace = True) #drops null values 
height_difference = pd.DataFrame({'Building Height' : height_difference1})
height_difference = height_difference.rename_axis(index='ID')

In [11]:
#10. MERGE INTO FINAL DETAILED TABLE#

out_final = shapefile.join(height_difference)
out_final = out_final.dropna(how = 'any', inplace = False)
out_final['Building ID'] = out_final.index
out_final['Roof Height'] = building_height['Roof Height']
out_final['Ground Level'] = ground_height['Ground Level']

In [13]:
#11. CSV EXTRACTION#

out_final.to_csv('height.csv',index = True)

In [133]:
#11. SHP EXTRACTION#

out_final.to_file("height_out_with_details.shp")

In [13]:
#analysis of lidar points collected on building tops#
building_summary = buildings.groupby('index_right')['Z coordinate'].describe()

In [14]:
#analysis of lidar points collected on ground buffer zone#
ground_summary = border.groupby('index_right')['Z coordinate'].describe()